In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb

In [2]:
import sys
import math

In [3]:
from keras.datasets import fashion_mnist
import sklearn
from sklearn.model_selection import train_test_split

2025-03-09 10:48:01.065248: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 10:48:01.074152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741497481.085180   59529 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741497481.088905   59529 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 10:48:01.101331: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
(train_data,train_label),(test_data,test_label) = fashion_mnist.load_data()
class_names = ["T-shirt", "Trouser", "Pullover", "Dress", "Coat", 
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle_boot"]

In [5]:


train_data, train_label = sklearn.utils.shuffle(train_data, train_label)
test_data, test_label = sklearn.utils.shuffle(test_data, test_label)

train_data = train_data.reshape(train_data.shape[0],-1)
test_data = test_data.reshape(test_data.shape[0],-1)
train_data = train_data/255.0
test_data = test_data/255.0
train_label = np.eye(10)[train_label]
test_label = np.eye(10)[test_label]

train_data,val_data,train_label,val_label = train_test_split(train_data,train_label,test_size=0.1,random_state=42)


In [76]:
class Neural_net:
    def __init__(self,num_hidden_layers,num_neurons_each_layer,activation_function,input_size,type_of_init,L2reg_const=0):
        self.num_hidden_layers = num_hidden_layers
        self.num_neurons_each_layer = num_neurons_each_layer
        self.input_size = input_size
        self.W,self.B = self.init_parameters(type_of_init)
        self.activation_function = activation_function
        self.L2reg_const = L2reg_const
        

    def init_parameters(self,init):
        W = []
        B = []
        if init == 'xavier':
            W.append(np.random.randn(self.num_neurons_each_layer[0], self.input_size) * np.sqrt(1 / self.input_size))
            for i in range(1,self.num_hidden_layers):
                W.append(np.random.randn(self.num_neurons_each_layer[i], self.num_neurons_each_layer[i-1]) * np.sqrt(1 / self.num_neurons_each_layer[i-1]))
        else:
            W.append(np.random.randn(self.num_neurons_each_layer[0], self.input_size) * 0.01)
            for i in range(1, self.num_hidden_layers):
                W.append(np.random.randn(self.num_neurons_each_layer[i], self.num_neurons_each_layer[i - 1]) * 0.01)

        B.append(np.zeros(shape=(1,self.num_neurons_each_layer[0])))
        for i in range(1,self.num_hidden_layers):
            B.append(np.zeros(shape=(1,self.num_neurons_each_layer[i])))

        return W, B

    def activation(self,x):
        if self.activation_function == 'ReLU':
            return self.ReLU(x)
        elif self.activation_function == 'tanh':
            return self.tanh(x)
        elif self.activation_function == 'sigmoid':
            return self.sigmoid(x)

    def activation_derivative(self,x):
        if self.activation_function == 'ReLU':
            return self.ReLU_derivative(x)
        elif self.activation_function == 'tanh':
            return self.tanh_derivative(x)
        elif self.activation_function == 'sigmoid':
            return self.sigmoid_derivative(x)

    def tanh(self,x):
        return np.array([((np.exp(z) - np.exp(-z))/((np.exp(z) + np.exp(-z)))) for z in x])

    def tanh_derivative(self,x):
        return np.array(1 - self.tanh(x)**2)
    
    def softmax_activation(self,x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def ReLU(self,x):
        return np.maximum(0,x)

    def ReLU_derivative(self,x):
        return np.where(x > 0, 1, 0)
    
    def sigmoid(self,x):
        x = np.clip(x, -500, 500)
        return 1/(1+np.exp(-x))

    def sigmoid_derivative(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))

    def FeedForward(self,data):
        a_l = []
        h_l = []
        input_linear_out = np.dot(data,self.W[0].T) + self.B[0]
        a_l.append(input_linear_out)
        for i in range(1,self.num_hidden_layers):
            activation_out = self.activation(a_l[-1])
            h_l.append(activation_out)
            linear_out = np.dot(h_l[-1],self.W[i].T) + self.B[i]
            a_l.append(linear_out)


        y_hat = self.softmax_activation(a_l[-1])
        return a_l,h_l,y_hat

    def val_loss_and_acc(self,val_data,val_label):
        acc = 0
        error = 0
        val_loss = []
        val_acc = []
        
        for d in range(val_data.shape[0]):
            a_li,h_li,y_hat_i = self.FeedForward(val_data[d].reshape(1,-1))
            s = [x.sum() for x in self.W]
            error += -np.sum(val_label[d] * np.log(y_hat_i[0])) + self.L2reg_const/2*sum(s)
            if np.argmax(val_label[d]) == np.argmax(y_hat_i[0]):
              acc += 1
        return error/val_data.shape[0],acc/val_data.shape[0]

    def backpropagation(self,A,H,y_hat,y,data):
      dA = []
      dH = []
      dW = []
      dB = []
      dA.append(-(y - y_hat))
      for i in range(self.num_hidden_layers-1,-1,-1):
        if i == self.num_hidden_layers - 1:
            grad_val_w = np.dot(dA[-1].reshape(-1, 1), H[-1].reshape(1, -1))
        elif i == 0:
          grad_val_w = np.dot(dA[-1].reshape(-1,1),data.reshape(1,-1))
        else:
          grad_val_w = np.dot(dA[-1].reshape(-1,1),H[i-1].reshape(1,-1))
        dW.append(grad_val_w)
        grad_val_b = np.sum(dA[-1],axis=0,keepdims=True)
        dB.append(grad_val_b)

        if i > 0:
          grad_L_hi = np.dot(self.W[i].T,dA[-1].T)
          grad_L_ai = np.multiply(grad_L_hi.T,self.activation_derivative(A[i-1]))
          dA.append(grad_L_ai)
      return dW[::-1],dB[::-1]

    def train(self,optimizer,epochs,learning_rate,data,label,val_data,val_label,batch_size=32,**kwargs):
        num_batches = len(data)//batch_size
        optimizer_config = {'learning_rate':learning_rate,**kwargs}
        if optimizer == 'momentum':
            momentum_W = [np.zeros_like(w) for w in self.W]
            momentum_B = [np.zeros_like(b) for b in self.B]
            optimizer_config['momentum_W'] = momentum_W
            optimizer_config['momentum_B'] = momentum_B
        elif optimizer == 'nesterov':
            momentum_W = [np.zeros_like(w) for w in self.W]
            momentum_B = [np.zeros_like(b) for b in self.B]
            optimizer_config['momentum_W'] = momentum_W
            optimizer_config['momentum_B'] = momentum_B
        elif optimizer == 'RMSprop':
            v_W = [np.zeros_like(w) for w in self.W]
            v_B = [np.zeros_like(b) for b in self.B]
            optimizer_config['v_W'] = v_W
            optimizer_config['v_B'] = v_B
        elif optimizer == 'adam' or optimizer == "nadam":
            momentum1_W = [np.zeros_like(w) for w in self.W]
            momentum1_B = [np.zeros_like(b) for b in self.B]
            momentum2_W = [np.zeros_like(w) for w in self.W]
            momentum2_B = [np.zeros_like(b) for b in self.B]
            optimizer_config['momentum1_W'] = momentum1_W
            optimizer_config['momentum1_B'] = momentum1_B
            optimizer_config['momentum2_W'] = momentum2_W
            optimizer_config['momentum2_B'] = momentum2_B
            optimizer_config['t'] = 0

        for ep in range(epochs):
            train_loss = 0
            train_acc = 0
            for batch in range(num_batches):
                start = batch*batch_size
                end = (batch+1)*batch_size
                batch_data = data[start:end]
                batch_label = label[start:end]
                batch_dw = [np.zeros_like(w) for w in self.W]
                batch_db = [np.zeros_like(b) for b in self.B]
                batch_correct = 0
                if optimizer in ('adam','nadam'):
                    optimizer_config['t'] += 1

                for i in range(batch_size):
                    a_li,h_li,y_hat_i = self.FeedForward(batch_data[i].reshape(1,-1))
                    s = [x.sum() for x in self.W]
                    train_loss += -np.sum(batch_label[i] * np.log(y_hat_i[0])) + self.L2reg_const/2*sum(s)
                    # train_loss += -np.sum(batch_label[i].reshape(1, -1) * np.log(y_hat_i[0].reshape(1, -1))) + self.L2reg_const/2*sum(s)

                    if np.argmax(batch_label[i]) == np.argmax(y_hat_i[0]):
                        batch_correct += 1
                    dw, db = self.backpropagation(a_li, h_li, y_hat_i[0], batch_label[i], batch_data[i])
                    for k in range(self.num_hidden_layers):
                        batch_dw[k] += dw[k]
                        batch_db[k] += db[k]
                train_acc += batch_correct
                self.W,self.B,optimizer_config = self.apply_optimizer(optimizer,batch_dw,batch_db,optimizer_config,batch_size)
                if (batch + 1) % 10 == 0:
                    val_loss, val_acc = self.val_loss_and_acc(val_data, val_label)
                    wandb.log({'epoch': ep + 1, 'train_loss': train_loss / ((batch+1)*batch_size), 'train_acc': train_acc / ((batch+1)*batch_size), 'val_loss': val_loss, 'val_acc': val_acc})
                    sys.stdout.write(f"\rEpoch {ep + 1}/{epochs} - Batch {batch + 1}/{num_batches} - Loss: {train_loss / ((batch+1)*batch_size):.6f} Train-Acc: {train_acc / ((batch+1)*batch_size):.6f} val-loss:{val_loss:.6f} val-Acc: {val_acc:.6f} ")
                    sys.stdout.flush()
            print()
    def apply_optimizer(self,optimizer,batch_dw,batch_db,config,batch_size):
        learning_rate = config['learning_rate']
        if optimizer == 'sgd':
            for k in range(self.num_hidden_layers):
                self.W[k] -= (learning_rate/batch_size)*batch_dw[k]
                self.B[k] -= (learning_rate/batch_size)*batch_db[k]
        elif optimizer == 'momentum':
            momentum_W = config['momentum_W']
            momentum_B = config['momentum_B']
            beta = config.get('beta',0.6)
            for k in range(self.num_hidden_layers):
                momentum_W[k] = beta*momentum_W[k] + batch_dw[k]
                momentum_B[k] = beta*momentum_B[k] + batch_db[k]
                self.W[k] -= learning_rate*momentum_W[k]
                self.B[k] -= learning_rate*momentum_B[k]
            config['momentum_W'] = momentum_W
            config['momentum_B'] = momentum_B
        elif optimizer == 'nesterov':
            momentum_W = config['momentum_W']
            momentum_B = config['momentum_B']
            beta = config.get('beta',0.6)
            for k in range(self.num_hidden_layers):
                W_lookahead = [w - beta * v for w, v in zip(self.W, momentum_W)]
                B_lookahead = [b - beta * v for b, v in zip(self.B, momentum_B)]
                
                momentum_W[k] = beta * momentum_W[k] + (learning_rate / batch_size) * batch_dw[k]
                momentum_B[k] = beta * momentum_B[k] + (learning_rate / batch_size) * batch_db[k]
                self.W[k] = W_lookahead[k] - momentum_W[k]
                self.B[k] = B_lookahead[k] - momentum_B[k]

            config['momentum_W'] = momentum_W
            config['momentum_B'] = momentum_B

        elif optimizer == 'RMSprop':
            v_W = config['v_W']
            v_B = config['v_B']
            beta = config.get('beta', 0.9)
            eps = config.get('eps', 1e-8)
            for k in range(self.num_hidden_layers):
                v_W[k] = beta * v_W[k] + (1 - beta) * (batch_dw[k]**2)
                v_B[k] = beta * v_B[k] + (1 - beta) * (batch_db[k]**2)

                adaptive_lr_w = (learning_rate / (np.sqrt(v_W[k]) + eps))
                adaptive_lr_b = (learning_rate / (np.sqrt(v_B[k]) + eps))
                self.W[k] -= adaptive_lr_w * batch_dw[k]
                self.B[k] -= adaptive_lr_b * batch_db[k]

            config['v_W'] = v_W
            config['v_B'] = v_B
        elif optimizer == 'adam':
            momentum1_W = config['momentum1_W']
            momentum1_B = config['momentum1_B']
            momentum2_W = config['momentum2_W']
            momentum2_B = config['momentum2_B']
            t = config['t']
            beta1 = config.get('beta1', 0.9)
            beta2 = config.get('beta2', 0.999)
            eps = config.get('eps', 1e-8)

            for i in range(self.num_hidden_layers):
                momentum1_W[i] = beta1*momentum1_W[i] + (1-beta1)*(batch_dw[i])
                momentum1_B[i] = beta1*momentum1_B[i] + (1-beta1)*(batch_db[i])

                momentum2_W[i] = beta2*momentum2_W[i] + (1-beta2)*(batch_dw[i]**2)
                momentum2_B[i] = beta2*momentum2_B[i] + (1-beta2)*(batch_db[i]**2)

                momentum1_W_hat = momentum1_W[i]/(1-(beta1**t))
                momentum1_B_hat = momentum1_B[i]/(1-(beta1**t))

                momentum2_W_hat = momentum2_W[i]/(1-(beta2**t))
                momentum2_B_hat = momentum2_B[i]/(1-(beta2**t))

                adaptive_lr_W = learning_rate/(np.sqrt(momentum2_W_hat) + eps)
                adaptive_lr_B = learning_rate/(np.sqrt(momentum2_B_hat) + eps)

                self.W[i] -= adaptive_lr_W * momentum1_W_hat
                self.B[i] -= adaptive_lr_B * momentum1_B_hat
            config['momentum1_W'] = momentum1_W
            config['momentum1_B'] = momentum1_B
            config['momentum2_W'] = momentum2_W
            config['momentum2_B'] = momentum2_B
            config['t'] = t
        elif optimizer == 'nadam':
            momentum1_W = config['momentum1_W']
            momentum1_B = config['momentum1_B']
            momentum2_W = config['momentum2_W']
            momentum2_B = config['momentum2_B']
            t = config['t']
            beta1 = config.get('beta1', 0.9)
            beta2 = config.get('beta2', 0.999)
            eps = config.get('eps', 1e-8)

            for i in range(self.num_hidden_layers):
                momentum1_W[i] = beta1 * momentum1_W[i] + (1 - beta1) * (batch_dw[i])
                momentum1_B[i] = beta1 * momentum1_B[i] + (1 - beta1) * (batch_db[i])

                momentum2_W[i] = beta2 * momentum2_W[i] + (1 - beta2) * (batch_dw[i]**2)
                momentum2_B[i] = beta2 * momentum2_B[i] + (1 - beta2) * (batch_db[i]**2)

                momentum1_W_hat = momentum1_W[i] / (1 - (beta1**t))
                momentum1_B_hat = momentum1_B[i] / (1 - (beta1**t))

                momentum2_W_hat = momentum2_W[i] / (1 - (beta2**t))
                momentum2_B_hat = momentum2_B[i] / (1 - (beta2**t))

                m_nestrov_W = beta1 * momentum1_W_hat + ((1 - beta1) * batch_dw[i])/(1-beta1**t)
                m_nestrov_B = beta1 * momentum1_B_hat + ((1 - beta1) * batch_db[i])/(1-beta1**t)

                adaptive_lr_W = learning_rate / (np.sqrt(momentum2_W_hat) + eps)
                adaptive_lr_B = learning_rate / (np.sqrt(momentum2_B_hat) + eps)

                self.W[i] -= adaptive_lr_W * m_nestrov_W
                self.B[i] -= adaptive_lr_B * m_nestrov_B
            config['momentum1_W'] = momentum1_W
            config['momentum1_B'] = momentum1_B
            config['momentum2_W'] = momentum2_W
            config['momentum2_B'] = momentum2_B
            config['t'] = t

        return self.W,self.B,config

In [58]:
nn = Neural_net(num_hidden_layers=4, num_neurons_each_layer=[128, 64,32,10],activation_function='ReLU' ,input_size=784,type_of_init='xavier',L2reg_const=0.0005)
optimizer = "adam"  # or "sgd", "momentum", "nesterov", "RMSprop", "nadam",'adam'
learning_rate = 0.0001
batch_size = 32
epochs = 10

In [59]:
optimizer_kwargs = {}
if optimizer == "momentum" or optimizer == "nesterov":
    optimizer_kwargs['beta'] = 0.9
elif optimizer == "RMSprop":
    optimizer_kwargs['beta'] = 0.8
    optimizer_kwargs['eps'] = 1e-8
elif optimizer == "adam" or optimizer == "nadam":
    optimizer_kwargs['beta1'] = 0.9
    optimizer_kwargs['beta2'] = 0.999
    optimizer_kwargs['eps'] = 1e-8

In [77]:
sweep_config = {
    'method': 'random',
    'metric': {'goal': 'maximize', 'name': 'val_acc'},
    'parameters': {
        'activation_function': {'values': ['sigmoid', 'tanh', 'ReLU']},
        'batch_size': {'values': [16, 32, 64]},
        'epochs': {'values': [5, 10]},
        'architecture': {
            'values': [
                {'num_of_hidden_layers': 3, 'hidden_layer_size': [128, 64, 10]},
                {'num_of_hidden_layers': 4, 'hidden_layer_size': [256, 128, 64, 10]},
                {'num_of_hidden_layers': 5, 'hidden_layer_size': [512, 256, 128, 64, 10]},
                {'num_of_hidden_layers': 6, 'hidden_layer_size': [1024, 512, 256, 128, 64, 10]}
            ]
        },
        'learning_rate': {'values': [1e-3,1e-4]},
        'optimizer': {'values': ['sgd', 'momentum', 'nesterov', 'RMSprop', 'adam', 'nadam']},
        'weight_initialization': {'values': ['random', 'xavier']},
        'weight_decay': {'values': [0, 0.0005,0.5]},
    }
}


In [78]:
def train():
    var1 = wandb.init()
    config = var1.config

    arch = config.architecture
    # Set up the model based on sweep config
    num_hidden_layers = arch['num_of_hidden_layers']
    num_neurons_each_layer = arch['hidden_layer_size']
    
    nn = Neural_net(
        num_hidden_layers=num_hidden_layers,
        num_neurons_each_layer=num_neurons_each_layer,
        activation_function = config.activation_function,
        input_size=784,
        type_of_init = config.weight_initialization,
        L2reg_const = config.weight_decay
    )
    
    nn.train(
        optimizer=config.optimizer,
        epochs=config.epochs,
        learning_rate=config.learning_rate,
        data=train_data,  # Make sure to define train_data and labels
        label=train_label,
        val_data=val_data,
        val_label=val_label,
        batch_size=config.batch_size
    )

In [63]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: cs24s031 (cs24s031-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [79]:
sweep_id = wandb.sweep(sweep_config,project='dl-assignment1-experiment')

Create sweep with ID: lyox2jab
Sweep URL: https://wandb.ai/cs24s031-indian-institute-of-technology-madras/dl-assignment1-experiment/sweeps/lyox2jab


In [80]:
wandb.agent(sweep_id,train,count=10)

wandb: Agent Starting Run: c4pw1x1l with config:
wandb: 	activation_function: sigmoid
wandb: 	architecture: {'hidden_layer_size': [512, 256, 128, 64, 10], 'num_of_hidden_layers': 5}
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0003
wandb: 	optimizer: nesterov
wandb: 	weight_decay: 0
wandb: 	weight_initialization: random


Epoch 1/5 - Batch 90/3375 - Loss: 2.303133 Train-Acc: 0.108333 val-loss:2.302826 val-Acc: 0.103667 

wandb: Ctrl + C detected. Stopping sweep.


Epoch 1/5 - Batch 120/3375 - Loss: 2.303806 Train-Acc: 0.106771 val-loss:2.302791 val-Acc: 0.103667 